In [1]:
import pandas as pd
# 1.Import an event log (csv format only) as a pandas dataframe
hospital = pd.read_csv('hospital_billing.csv')
print(hospital)

       Unnamed: 0 case_id activity lifecycle resource            timestamp  \
0               1       A      NEW  complete     ResA  2012-12-16 19:33:10   
1               2       A      FIN  complete      NaN  2013-12-15 19:00:37   
2               3       A  RELEASE  complete      NaN  2013-12-16 03:53:38   
3               4       A  CODE OK  complete      NaN  2013-12-17 12:56:29   
4               5       A   BILLED  complete     ResB  2013-12-19 03:44:31   
...           ...     ...      ...       ...      ...                  ...   
49946       49947     PTN      NEW  complete     ResA  2013-03-27 22:29:13   
49947       49948     PTN      FIN  complete      NaN  2013-05-27 00:43:02   
49948       49949     PTN  RELEASE  complete      NaN  2013-05-27 01:56:12   
49949       49950     PTN  CODE OK  complete      NaN  2013-05-30 01:42:39   
49950       49951     PTN   BILLED  complete     ResB  2013-08-27 13:13:01   

      actorange actred blocked casetype  ... iscancelled isclos

In [3]:
#2.Rename the attributes as “caseid, activity, ts” if names are different (ts is for timestamp!)
hospital.rename(columns={'case_id': 'caseid', 'activity': 'activity', 'timestamp': 'ts'}, inplace=True)
print(hospital)

       Unnamed: 0 caseid activity lifecycle resource                   ts  \
0               1      A      NEW  complete     ResA  2012-12-16 19:33:10   
1               2      A      FIN  complete      NaN  2013-12-15 19:00:37   
2               3      A  RELEASE  complete      NaN  2013-12-16 03:53:38   
3               4      A  CODE OK  complete      NaN  2013-12-17 12:56:29   
4               5      A   BILLED  complete     ResB  2013-12-19 03:44:31   
...           ...    ...      ...       ...      ...                  ...   
49946       49947    PTN      NEW  complete     ResA  2013-03-27 22:29:13   
49947       49948    PTN      FIN  complete      NaN  2013-05-27 00:43:02   
49948       49949    PTN  RELEASE  complete      NaN  2013-05-27 01:56:12   
49949       49950    PTN  CODE OK  complete      NaN  2013-05-30 01:42:39   
49950       49951    PTN   BILLED  complete     ResB  2013-08-27 13:13:01   

      actorange actred blocked casetype  ... iscancelled isclosed msgcode  

In [4]:
#3.Create an event log (= a new dataframe) retaining only the caseid, activity and ts attributes
hospital = hospital[['caseid', 'activity','ts']]
print(hospital)

      caseid activity                   ts
0          A      NEW  2012-12-16 19:33:10
1          A      FIN  2013-12-15 19:00:37
2          A  RELEASE  2013-12-16 03:53:38
3          A  CODE OK  2013-12-17 12:56:29
4          A   BILLED  2013-12-19 03:44:31
...      ...      ...                  ...
49946    PTN      NEW  2013-03-27 22:29:13
49947    PTN      FIN  2013-05-27 00:43:02
49948    PTN  RELEASE  2013-05-27 01:56:12
49949    PTN  CODE OK  2013-05-30 01:42:39
49950    PTN   BILLED  2013-08-27 13:13:01

[49951 rows x 3 columns]


In [5]:
#4.Create a new dataframe with columns caseid, list of events in a case (ordered by timestamp)
def create_event_list(event_log):
    event_log['ts'] = pd.to_datetime(event_log['ts'] )#convert ts into a python datetime object
    grouped=event_log.sort_values('ts').groupby('caseid')['activity'].apply(list).reset_index()#sort and group
    grouped.columns = ['caseid', 'list_ordered_ts']
    return grouped
events_hospital = create_event_list(hospital)
print(events_hospital)

     caseid                                    list_ordered_ts
0         A               [NEW, FIN, RELEASE, CODE OK, BILLED]
1        AA  [NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...
2       AAA                                              [NEW]
3       AAB                                              [NEW]
4       AAC  [NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...
...     ...                                                ...
9994    ZZI       [NEW, FIN, RELEASE, CODE OK, REOPEN, DELETE]
9995    ZZJ                                              [NEW]
9996    ZZK  [NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...
9997    ZZL               [NEW, FIN, RELEASE, CODE OK, BILLED]
9998    ZZM  [NEW, CHANGE DIAGN, FIN, RELEASE, CODE OK, BIL...

[9999 rows x 2 columns]


In [6]:
#5.	Augment the event log created at 4 with two new attributes: duration, event_number. duration = ts(last event in case) - ts(first event in case);Event_number: number of events in a case
def augment_event_log(event_log):
    event_log['ts'] = pd.to_datetime(event_log['ts'])
    augmented_df = event_log.groupby('caseid').agg(duration=('ts', lambda x: x.max() - x.min()), event_number=('activity', 'count')).reset_index()
    return augmented_df

augmented_event_hospital = augment_event_log(hospital)
print(augmented_event_hospital)

     caseid          duration  event_number
0         A 367 days 08:11:21             5
1        AA 193 days 10:31:09             6
2       AAA   0 days 00:00:00             1
3       AAB   0 days 00:00:00             1
4       AAC 186 days 12:20:43             6
...     ...               ...           ...
9994    ZZI 117 days 18:41:22             6
9995    ZZJ   0 days 00:00:00             1
9996    ZZK 116 days 11:07:50             6
9997    ZZL 158 days 07:16:47             5
9998    ZZM 158 days 10:52:19             6

[9999 rows x 3 columns]


In [8]:
#6.Find the longest/shortest case by duration - duration = ts(last event in case) - ts(first event in case)
def find_extreme_cases_duration(augmented_event_log):
    longest_case = augmented_event_log[augmented_event_log['duration'] == augmented_event_log['duration'].max()]
    shortest_case = augmented_event_log[augmented_event_log['duration'] == augmented_event_log['duration'].min()]
    return longest_case, shortest_case

longest_case_hospital, shortest_case_hospital = find_extreme_cases_duration(augmented_event_hospital)
print(longest_case_hospital, shortest_case_hospital)

     caseid           duration  event_number
2599    GTC 1035 days 10:06:32             6      caseid duration  event_number
2       AAA   0 days             1
3       AAB   0 days             1
5       AAD   0 days             1
12      AAK   0 days             1
27      ABK   0 days             1
...     ...      ...           ...
9983    ZYL   0 days             1
9985     ZZ   0 days             1
9986    ZZA   0 days             1
9988    ZZC   0 days             1
9995    ZZJ   0 days             1

[1836 rows x 3 columns]


In [10]:
#7.Find the longest case(s) by number of events
def find_longest_case(augmented_event_log):
    longest_case = augmented_event_log[augmented_event_log['event_number'] == augmented_event_log['event_number'].max()]
    return longest_case

longest_case_hospital = find_longest_case(augmented_event_hospital)
print(longest_case_hospital)

     caseid          duration  event_number
4648    MBL 375 days 01:52:29           217
